In [ ]:
import os
import pandas as pd
import time


start = time.time()

version = "knu_score" #"new_dict_score"
top = 
qlist = ["여성", "여자+-여성", "남성", '남자+-남성']

pos_score, neg_score = list(), list()
df_pos = pd.DataFrame(columns=['title', 'source', 'date', 'text', version])
df_neg = pd.DataFrame(columns=['title', 'source', 'date', 'text', version])
df_pos_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', '#3_'+version, f"{version+'_pos'}.xlsx")
df_neg_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', '#3_'+version, f"{version+'_neg'}.xlsx")

for query in qlist:
    for year in range(2000, 2020):
        for month in range(1, 13):
            xlsx_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', '#3_'+version, "#3_" + query, 
                                     f"{'#3_'+str(year)+'_'+str(month).zfill(2)+'_'+query}.xlsx")

            df = pd.read_excel(xlsx_name, index_col=0)
            df = df.drop(columns=['url'])
            for i in range(len(df)):

                d = df.loc[i]
                k = d['knu_score']

                if k > 0 and k >= min(pos_score):
                    pos_score.append(k)
                    df_pos.append(d)
                if len(pos_score) > top:
                    ind = pos_score.index(min(pos_score))
                    pos_score.pop(ind)
                    df_pos.drop(ind)
                    df_pos.reset_index(drop=True)

                if k < 0 and k =< max(neg_score):
                    neg_score.append(k)
                    df_neg.append(d)
                if len(neg_score) > top:
                    ind = neg_score.index(max(neg_score))
                    neg_score.pop(ind)
                    df_neg.drop(ind)
                    df_neg.reset_index(drop=True)
                            
df_pos.to_excel(df_pos_name)
df_neg.to_excel(df_neg_name)

end = time.time()
print(end - start)